In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND, MIND_news
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 1000
config.mode = 'dev'
# config.scale = 'large'
# config.fast = True
config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
# manager.gather_same_user_impr()

a = MIND(manager, file_directory)
# b = MIND_news(manager)

[2021-10-27 09:27:59,253] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDlarge_dev/1000/behaviors.pkl
[2021-10-27 09:28:02,916] INFO (utils.MIND) rank 0 extracting users who browsed news 21563
[2021-10-27 09:28:03,030] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDlarge_dev/news.pkl
[2021-10-27 09:28:03,467] INFO (utils.utils) deduplicating...


In [ ]:
# manager.construct_whole_dataset()

In [8]:
a[3]['his_encoded_index']

array([[  101,  2129,  5180, ...,     0,     0,     0],
       [  101,  2450,  4858, ...,     0,     0,     0],
       [  101,  3419, 10184, ...,  2074,  2917, 26077],
       ...,
       [  100,   100,   100, ...,   100,   100,   100],
       [  100,   100,   100, ...,   100,   100,   100],
       [  100,   100,   100, ...,   100,   100,   100]])

In [ ]:
r = a[0]

In [ ]:
r['his_attn_mask'].shape

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDlarge_test/bm25.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

index = 37405
news = t.convert_ids_to_tokens(a.encoded_news_original[index])
mask = a.attn_mask_original[index]
subword = a.subwords_original[index]

for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)